<a href="https://www.kaggle.com/code/bharathkumar1011/resume-parse-hf-llama-3-2-3b?scriptVersionId=219013160" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# First, install required packages
!pip install langchain langchain-community langchain-core langchain_huggingface transformers pdfplumber accelerate --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 105.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 106.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 86.4 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

from langchain_community.document_loaders import PDFPlumberLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate

In [3]:
def extract_text_pdf(pdf_path):
        loader = PDFPlumberLoader(pdf_path)
        docs = loader.load()
        text = ''
        for doc in docs:
            text += doc.page_content # Append extracted text
        return text 


In [4]:
system_prompt = """You are an AI assistant designed to extract structured resume data.
                   Always respond with a strictly valid JSON object. Use `null` for missing values,
                   ensuring compliance with JSON standards. Do not include explanations,
                   comments, or any additional text outside the JSON structure.
                """



In [5]:
# human_prompt = """{
#   "instruction": "You are an expert assistant for parsing resumes into structured JSON format.",
#   "task": "Extract key details from the resume text provided below and output a valid JSON object with the following fields:",
#   "fields": [
#     "Name",
#     "Email ID",
#     "Skills",
#     "Education",
#     "Experience",
#   ],
#   "notes": [
#     "Use `null` if a field is not found.",
#     "Ensure the JSON is valid and well-structured.",
#     "Do not include any explanations or extra text."
#   ],
#   "example": {
#     "Name": "John Doe",
#     "Contact Information": "john.doe@example.com, +1-234-567-890",
#     "Summary": "Experienced software developer specializing in AI solutions.",
#     "Skills": ["Python", "Machine Learning", "NLP", "Django"],
#     "Education": "B.Sc. in Computer Science, XYZ University",
#     "Experience": [
#       {
#         "Company": "Tech Solutions Inc.",
#         "Role": "Software Developer",
#         "Duration": "Jan 2020 - Dec 2022",
#         "Responsibilities": ["Developed web applications", "Integrated AI models"]
#       }
#     ],
#     "Certifications": ["AWS Certified Solutions Architect"],
#     "Projects": ["Resume Parsing Project using LLaMA and LangChain"],
#     "Languages": ["English", "Spanish"],
#     "Achievements": ["Employee of the Month (March 2021)"]
#   }
# }"""


In [6]:
human_prompt = """
             **Task:** Extract key information from the following resume text.

            **Resume Text:**
            {context}

            **Instructions:**
            Please extract the following information and format it in a clear structure:

            1. **Contact Information:**
            - Name:
            - Email:
            - Phone Number:
            - Website/Portfolio/LinkedIn:
            - Github Profile:

            2. **Education:**
            - Institution Name:
            - Degree:
            - Graduation Date:

            3. **Experience:**
            - Job Title:
            - Company Name:
            - Location:
            - Dates of Employment:
            - Description:

            5. **Skills:**
            - Skills:

            **Question:**
            Extract this information as a structured and valid JSON object. Use `null` for missing or unavailable values.
        """


In [7]:
pdf_path = "/kaggle/input/test-resumes/Steve Sun - Resume (2).pdf"

In [8]:
context = extract_text_pdf(pdf_path)
print(context)

Steve Sun
Results-driven professional with expertise in data analysis, machine learning, and statistical modeling.
Passionate about transforming complex data into actionable insights to drive innovation and business
impact.
stevesun1245@gmail.com (123)456789 Hyderabad linkedin.com/in/stevesun
WORK EXPERIENCE
DataNova Insights
Data Scientist March 2021 – Present
• Developed and deployed machine learning models to optimize customer retention, increasing
engagement by 15%.
• Conducted exploratory data analysis (EDA) and statistical testing to derive actionable business
insights.
• Built scalable ETL pipelines to process large datasets using Python and SQL.
• Collaborated with cross-functional teams to improve decision-making through predictive analytics.
Neural Sphere Labs
Data Scientist June 2019 – December 2023
• Designed and implemented deep learning models for image classification, improving accuracy by
20%.
• Automated data preprocessing and feature engineering workflows using Python

In [9]:
template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt),
])

In [10]:
question = "Extract key information from this resume."
complete_prompt = template.format_messages(context=context, question=question)


In [21]:
complete_prompt

[SystemMessage(content='You are an AI assistant designed to extract structured resume data.\n                   Always respond with a strictly valid JSON object. Use `null` for missing values,\n                   ensuring compliance with JSON standards. Do not include explanations,\n                   comments, or any additional text outside the JSON structure.\n                ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n             **Task:** Extract key information from the following resume text.\n\n            **Resume Text:**\n            Steve Sun\nResults-driven professional with expertise in data analysis, machine learning, and statistical modeling.\nPassionate about transforming complex data into actionable insights to drive innovation and business\nimpact.\nstevesun1245@gmail.com (123)456789 Hyderabad linkedin.com/in/stevesun\nWORK EXPERIENCE\nDataNova Insights\nData Scientist March 2021 – Present\n• Developed and deployed machine learning models t

### With the bullet point symbol in the complete prompt, the response is cutting off and not giving the whole answer. So, I replaced the bullet point symbol with space, which solved the JSON format structured output problem with the LLaMA invoke response.

# What I learned:
### -The complete prompt should not contain any bullet points or any type of cutting symbols to perform better.


In [26]:
# Assuming complete_prompt is a list of objects that have a 'content' attribute
complete_prompt = [text.content.replace("•", " ") for text in complete_prompt]

# Output the result
print(complete_prompt)


['You are an AI assistant designed to extract structured resume data.\n                   Always respond with a strictly valid JSON object. Use `null` for missing values,\n                   ensuring compliance with JSON standards. Do not include explanations,\n                   comments, or any additional text outside the JSON structure.\n                ', '\n             **Task:** Extract key information from the following resume text.\n\n            **Resume Text:**\n            Steve Sun\nResults-driven professional with expertise in data analysis, machine learning, and statistical modeling.\nPassionate about transforming complex data into actionable insights to drive innovation and business\nimpact.\nstevesun1245@gmail.com (123)456789 Hyderabad linkedin.com/in/stevesun\nWORK EXPERIENCE\nDataNova Insights\nData Scientist March 2021 – Present\n  Developed and deployed machine learning models to optimize customer retention, increasing\nengagement by 15%.\n  Conducted exploratory da

In [12]:
notebook_login()

In [14]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
task = "text-generation"

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [15]:
pipe = pipeline(
    task,
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
)

Device set to use cuda:0


In [16]:
llm = HuggingFacePipeline(pipeline=pipe)

In [27]:
%%time
response = llm.invoke(complete_prompt, skip_prompt=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


CPU times: user 16.8 s, sys: 4.1 ms, total: 16.8 s
Wall time: 16.7 s


In [28]:
print(response)




            {
                "name": "Steve Sun",
                "email": "stevesun1245@gmail.com",
                "phone": null,
                "website": null,
                "linkedin": "linkedin.com/in/stevesun",
                "github": null,
                "education": {
                    "institution": "Skyline University",
                    "degree": "Master of Science in Data Science",
                    "graduationDate": "May 2020"
                },
                "experience": [
                    {
                        "title": "Data Scientist",
                        "company": "DataNova Insights",
                        "location": "Hyderabad",
                        "dates": "March 2021 – Present",
                        "description": "Developed and deployed machine learning models to optimize customer retention, increasing engagement by 15%. Conducted exploratory data analysis (EDA) and statistical testing to derive actionable business insights